In [1]:
import pandas as pd
from time import sleep
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import sys

In [2]:
# tranform the transcript text to a dataframe
def transcript2df(transcript):
    pattern = r'^(([0-9]{2}|[0-9]{1}):([0-9]{2}|[0-9]{1})|([0-9]{2}|[0-9]{1})):[0-9]{2}$'
    transcript_timestamps,transcript_dialog,prev = [],[],None
    if not re.match(pattern, transcript[0]): # delete the title
        transcript.pop(0)
    for item in transcript:
        if re.match(pattern, item):
            transcript_timestamps.append(item)
            if prev == None: prev = item
            else: 
                transcript_dialog.append(prev)
                prev = item
        else:
            if re.match(pattern, prev):
                prev = item
            else:
                prev = prev + ' ' + item
    transcript_dialog.append(item)
    df = pd.DataFrame({'timestamp':transcript_timestamps, 
                   'text':transcript_dialog})    
    return df

In [3]:
# scrape video transcript
def get_youtube_transcript(video_ids,driver_='Chrome'):
    frames,not_found_video_id = [],[]
    if driver_ == 'Chrome': driver = webdriver.Chrome()
    elif driver_ == 'Firefox': driver = webdriver.Firefox()
    else: sys.exit('Driver not found')
    wait = WebDriverWait(driver, 10)
    for video_id in video_ids:
        try:
            driver.get(f"https://www.youtube.com/watch?v={video_id}")
            wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="button-shape"]/button'))).click()
            wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="items"]/ytd-menu-service-item-renderer[2]/tp-yt-paper-item/yt-formatted-string'))).click()
            transcript_element = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='body']/ytd-transcript-segment-list-renderer")))
            transcript = transcript_element.text
            transcript = transcript.split('\n')
            frames.append(transcript2df(transcript).assign(video_id=video_id))
        except:
            not_found_video_id.append(video_id)
    driver.quit()
    return pd.concat(frames,ignore_index=True),not_found_video_id

In [4]:
df = pd.read_parquet('../../data/scrap_oso_trava/playlist_videos.parquet')

In [5]:
print(len(df.id.unique()))
result,not_found_video_id = get_youtube_transcript(df.id.unique())
result.head()

222


,timestamp,text,video_id
0,0:00,Nosotros le mentíamos como que se iba a salvar...,fL9FNpuGK_8
1,0:06,describir mi familia era una familia enferma n...,fL9FNpuGK_8
2,0:14,brazos meto a Yuli me voy le llamo al médico l...,fL9FNpuGK_8
3,0:20,impedirte a llorar entre eso soy osos y silenc...,fL9FNpuGK_8
4,0:26,cerebral más joven un cáncer peor pronóstico p...,fL9FNpuGK_8


In [27]:
print(len(df.id.unique()))
print(len(result.video_id.unique()))
print(len(not_found_video_id))

222
216
6


In [28]:
result.to_parquet('../scripts/data/scrap_oso_trava/videos_transcript.parquet',index=False)